In [1]:
import sys
sys.path.append("../helper_functions")
import defillama_utils as dfl
sys.path.pop()
import requests as r
import pandas as pd
import plotly.express as px

chain = 'Base'

In [2]:
# dt = r.get('https://api.llama.fi/protocol/aave-v3').json()
# dt = dt['chainTvls']['Optimism']['tokens']
# dt = pd.json_normalize(dt)
# print(dt)

In [3]:
df_base = dfl.get_protocols_by_chain(chain)
df_base = df_base[['category','name','slug']]
df_base['chain'] = chain

In [4]:
tvl_dfs = []
for i, row in df_base.iterrows():
        try:
                df = dfl.get_single_tvl(row['slug'],[row['chain']])
                df = df.groupby(['date','chain','protocol','category']).agg({'total_app_tvl':'mean'})
                df.reset_index(inplace=True)
                df['name'] = row['name']
                tvl_dfs.append(df)
        except:
                continue

tvl_df = pd.concat(tvl_dfs)
tvl_df = tvl_df[tvl_df['total_app_tvl'] > 0]
tvl_df['date'] = pd.to_datetime(tvl_df['date'])
display(tvl_df)

ValueError: No objects to concatenate

In [ ]:
launch_df = tvl_df.groupby(['chain','protocol','category','name']).agg({'date':'min', 'total_app_tvl':'last'})
launch_df = launch_df.rename(columns={
        'date':'launch_date',
        'total_app_tvl':'latest_tvl_usd'
})
launch_df.reset_index(inplace=True)
display(launch_df)